In [190]:
import pandas as pd
import urllib.request
import json
import pandas as pd
from datetime import datetime
import seaborn as sns
cm = sns.light_palette("red", as_cmap=True)
from jinja2 import Environment, FileSystemLoader
from sqlalchemy import create_engine
from datetime import datetime
import pdfkit 
import json
import os

In [191]:
dfLoadPontosIda = pd.read_csv("historicoPesquisaPontosIda.csv") 
dfLoadPontosVolta = pd.read_csv("historicoPesquisaPontosVolta.csv")
dfGrafico = pd.read_csv("historicoPesquisaPrecos.csv") 


print(dfGrafico['DataPesquisa'].max())

print(dfGrafico['de'].unique())
print(dfGrafico['para'].unique())


print(dfLoadPontosIda['De'].unique())
print(dfLoadPontosIda['Para'].unique())
print(dfLoadPontosVolta['De'].unique())
print(dfLoadPontosVolta['Para'].unique())


dfGrafico['DataPesquisaReferencia'] = dfGrafico['DataPesquisa'].apply(lambda x: x[0:13])
dfLoadPontosIda['DataPesquisaReferencia'] = dfLoadPontosIda['DataPesquisa'].apply(lambda x: x[0:13])
dfLoadPontosVolta['DataPesquisaReferencia'] = dfLoadPontosVolta['DataPesquisa'].apply(lambda x: x[0:13])

dataMaximaPesquisa = dfGrafico.max()['DataPesquisa'][0:13]
dfGraficoAtual = dfGrafico.query('DataPesquisaReferencia == @dataMaximaPesquisa')

dataMaximaPesquisaPontoIda = dfLoadPontosIda.max()['DataPesquisa'][0:13]
dfGraficoPontoIdaAtual = dfLoadPontosIda.query('DataPesquisaReferencia == @dataMaximaPesquisaPontoIda')

dataMaximaPesquisaPontoVolta = dfLoadPontosVolta.max()['DataPesquisa'][0:13]
dfGraficoPontoVoltaAtual = dfLoadPontosVolta.query('DataPesquisaReferencia == @dataMaximaPesquisaPontoVolta')

print(dfLoadPontosIda.count()[0],'-',dfGraficoPontoIdaAtual.count()[0])
print(dfLoadPontosVolta.count()[0],'-',dfGraficoPontoVoltaAtual.count()[0])
print(dfGrafico.count()[0],'-',dfGraficoAtual.count()[0])


del dfGraficoAtual['Unnamed: 0']
del dfGraficoAtual['DataPesquisa']
del dfGraficoAtual['DataPesquisaReferencia']
del dfGraficoPontoIdaAtual['Unnamed: 0']
del dfGraficoPontoIdaAtual['DataPesquisa']
del dfGraficoPontoIdaAtual['DataPesquisaReferencia']
del dfGraficoPontoVoltaAtual['Unnamed: 0']
del dfGraficoPontoVoltaAtual['DataPesquisa']
del dfGraficoPontoVoltaAtual['DataPesquisaReferencia']


#display(dfLoadPontosIda.head(2))
#display(dfLoadPontosVolta.head(2))
#display(dfGrafico.head(2))


31/01/2020 09:01:49
['POA' 'GRU' 'GIG' 'BSB' 'FOR']
['ATL' 'MIA' 'MDZ' 'BRC' 'LIM' 'CTG' 'ADZ' 'FRA']
['POA' 'GRU' 'GIG']
['MIA' 'ATL' 'MDZ' 'BRC' 'LIM' 'CTG' 'ADZ' 'FRA']
['POA' 'GRU' 'GIG' 'MIA' 'LIM' 'MDZ']
['MIA' 'MDZ' 'ATL' 'BRC' 'LIM' 'CTG' 'ADZ' 'POA' 'GRU' 'GIG']
9746 - 1009
3460 - 74
8207 - 402


In [192]:
print(dfGraficoAtual['de'].unique())
print(dfGraficoAtual['para'].unique())

['POA' 'GRU' 'GIG']
['ADZ' 'FRA' 'ATL' 'MIA' 'MDZ' 'BRC' 'LIM' 'CTG']


In [193]:
for origem in dfGraficoAtual['de'].unique():    
    dfGraficoAtualP = pd.pivot_table(dfGraficoAtual.query('de==@origem'),values='preco',index=['de','para','Ida'],columns='Volta')
    dfGraficoAtualP.fillna(0,inplace=True)
    display(dfGraficoAtualP.style.background_gradient(cmap='OrRd'))

In [194]:
display(dfGraficoPontoIdaAtual.columns)
for origem in dfGraficoPontoIdaAtual['De'].unique():      
    dfGraficoPontoIdaAtualP = pd.pivot_table(dfGraficoPontoIdaAtual.query('De==@origem & pontos < 85000'),values=['pontos','preco'],index=['De','Para','PartidaData','PartidaHora'],columns='Paradas')
    dfGraficoPontoIdaAtualP.fillna(0,inplace=True)
    print('DE:',origem,dfGraficoPontoIdaAtualP.count())
    display(dfGraficoPontoIdaAtualP.style.background_gradient(cmap='OrRd'))

Index(['ChegadaData', 'ChegadaHora', 'ChegadaMinuto', 'De', 'Para', 'Paradas',
       'PartidaData', 'PartidaHora', 'PartidaMinuto', 'overnight', 'pontos',
       'preco', 'precoMenosSaldo'],
      dtype='object')

DE: POA         Paradas
pontos  0          75
        1          75
        2          75
preco   0          75
        1          75
        2          75
dtype: int64


DE: GRU         Paradas
pontos  0          62
        1          62
        2          62
preco   0          62
        1          62
        2          62
dtype: int64


DE: GIG         Paradas
pontos  1          51
        2          51
preco   1          51
        2          51
dtype: int64


In [195]:
display(dfGraficoPontoVoltaAtual.columns)
for origem in dfGraficoPontoVoltaAtual['De'].unique():  
    dfGraficoPontoVoltaAtualP = pd.pivot_table(dfGraficoPontoVoltaAtual.query('De==@origem & pontos < 185000'),values=['pontos'],index=['De','Para','PartidaData','PartidaHora'],columns='Paradas')
    dfGraficoPontoVoltaAtualP.fillna(0,inplace=True)
    print('DE:',origem,dfGraficoPontoIdaAtualP.count())
    display(dfGraficoPontoVoltaAtualP.style.background_gradient(cmap='OrRd'))

Index(['ChegadaData', 'ChegadaHora', 'ChegadaMinuto', 'De', 'Para', 'Paradas',
       'PartidaData', 'PartidaHora', 'PartidaMinuto', 'overnight', 'pontos',
       'preco', 'precoMenosSaldo'],
      dtype='object')

DE: MIA         Paradas
pontos  1          51
        2          51
preco   1          51
        2          51
dtype: int64


DE: LIM         Paradas
pontos  1          51
        2          51
preco   1          51
        2          51
dtype: int64


DE: MDZ         Paradas
pontos  1          51
        2          51
preco   1          51
        2          51
dtype: int64
